In [ ]:
!pip install transformers
!pip install datasets
!pip install transformers


In [ ]:
import torch, datasets, transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].replace({'positive':0, 'negative':1})
dataset = Dataset.from_pandas(df)

<ipython-input-41-8d7c80c5a659>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive':0, 'negative':1})


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["review"])
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'sentiment'])
print(type(tokenized_datasets['sentiment']))
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")

<class 'torch.Tensor'>


In [ ]:
print(tokenized_datasets['input_ids'].shape)
print(tokenized_datasets['attention_mask'].shape)
print(tokenized_datasets['labels'].shape)

torch.Size([50000, 1024])
torch.Size([50000, 1024])
<class 'torch.Tensor'>
torch.Size([50000])


In [ ]:
train_test = tokenized_datasets.train_test_split(test_size=0.2)  # 80% train, 20% test
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

In [ ]:
sentiment_scores = torch.tensor(df["sentiment"].values)  # Assuming this is your sentiment column
sentiment_scores
# Add sentiment scores to eval_dataset if needed
#eval_dataset = eval_dataset.add_column("sentiment", sentiment_scores.tolist())

tensor([0, 0, 0,  ..., 1, 1, 1])

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Fine_tuning_gpt2",
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bulina2003 (stefan_durlanescu-university-of-groningen). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.evaluate()